In [ ]:
!pip install pyspark

In [ ]:
from google.colab import drive
import pandas as pd 
import numpy as np
drive.mount('/content/drive',force_remount=True)
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Bank_classification').getOrCreate()

Mounted at /content/drive


In [ ]:
df = spark.read.csv('/content/drive/MyDrive/BIGDATA/Week5/bank.csv', header = True, inferSchema=True)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [ ]:
df = df.toPandas()
df.deposit = df.deposit.apply(lambda x: 1 if x == 'yes' else 0)

In [ ]:

class_name = 'deposit'
df.groupby(class_name).count()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
deposit,,,,,,,,,,,,,,,,
0,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873
1,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289


In [ ]:
df = pd.get_dummies(df)

df = spark.createDataFrame(df)
df.head()

Row(age=59, balance=2343, day=5, duration=1042, campaign=1, pdays=-1, previous=0, deposit=1, job_admin.=1, job_blue-collar=0, job_entrepreneur=0, job_housemaid=0, job_management=0, job_retired=0, job_self-employed=0, job_services=0, job_student=0, job_technician=0, job_unemployed=0, job_unknown=0, marital_divorced=0, marital_married=1, marital_single=0, education_primary=0, education_secondary=1, education_tertiary=0, education_unknown=0, default_no=1, default_yes=0, housing_no=0, housing_yes=1, loan_no=1, loan_yes=0, contact_cellular=0, contact_telephone=0, contact_unknown=1, month_apr=0, month_aug=0, month_dec=0, month_feb=0, month_jan=0, month_jul=0, month_jun=0, month_mar=0, month_may=1, month_nov=0, month_oct=0, month_sep=0, poutcome_failure=0, poutcome_other=0, poutcome_success=0, poutcome_unknown=1)

In [ ]:
from pyspark.ml.feature import StringIndexer
class_name = 'deposit'
indexer = StringIndexer()
indexer.setInputCol(class_name).setOutputCol('label')
df1 = indexer.fit(df).transform(df)
df1 = df1.withColumnRenamed("job_admin.", "job_admin")

In [ ]:
[training_data, test_data] = df1.randomSplit([0.8,0.2])

In [ ]:
training_data.toPandas()

,age,balance,day,duration,campaign,pdays,previous,deposit,job_admin,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,label
0,18,3,25,130,2,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0
1,18,108,8,169,1,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1.0
2,18,108,9,92,1,183,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1.0
3,18,608,12,267,1,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0
4,19,103,10,104,2,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8910,82,1381,28,86,3,93,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0.0
8911,83,4565,13,175,2,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.0
8912,85,1934,12,140,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.0
8913,87,433,15,150,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0


In [ ]:
class_name = 'label'
feature_names = df.columns[:-1]
print(class_name)
feature_names[8] = "job_admin"

print(feature_names)

label
['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'deposit', 'job_admin', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'marital_divorced', 'marital_married', 'marital_single', 'education_primary', 'education_secondary', 'education_tertiary', 'education_unknown', 'default_no', 'default_yes', 'housing_no', 'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular', 'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep', 'poutcome_failure', 'poutcome_other', 'poutcome_success']


In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler()
assembler.setInputCols(feature_names).setOutputCol('features')

transformed_data = assembler.transform(df1)

transformed_data.show()

+---+-------+---+--------+--------+-----+--------+-------+---------+---------------+----------------+-------------+--------------+-----------+-----------------+------------+-----------+--------------+--------------+-----------+----------------+---------------+--------------+-----------------+-------------------+------------------+-----------------+----------+-----------+----------+-----------+-------+--------+----------------+-----------------+---------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------------+--------------+----------------+----------------+-----+--------------------+
|age|balance|day|duration|campaign|pdays|previous|deposit|job_admin|job_blue-collar|job_entrepreneur|job_housemaid|job_management|job_retired|job_self-employed|job_services|job_student|job_technician|job_unemployed|job_unknown|marital_divorced|marital_married|marital_single|education_primary|education_secondary|education

In [ ]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])
print(training_data.show())

+---+-------+---+--------+--------+-----+--------+-------+---------+---------------+----------------+-------------+--------------+-----------+-----------------+------------+-----------+--------------+--------------+-----------+----------------+---------------+--------------+-----------------+-------------------+------------------+-----------------+----------+-----------+----------+-----------+-------+--------+----------------+-----------------+---------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------------+--------------+----------------+----------------+-----+--------------------+
|age|balance|day|duration|campaign|pdays|previous|deposit|job_admin|job_blue-collar|job_entrepreneur|job_housemaid|job_management|job_retired|job_self-employed|job_services|job_student|job_technician|job_unemployed|job_unknown|marital_divorced|marital_married|marital_single|education_primary|education_secondary|education

In [ ]:
from pyspark.ml.classification import LogisticRegression
model = LogisticRegression(featuresCol = 'features',labelCol=class_name, maxIter=30)

In [ ]:
M = model.fit(training_data)

In [ ]:
predictions = M.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(predictions))

Logistic Regression Accuracy: 1.0
